In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, SGD
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import timeit
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import load_model
import os


device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

2023-05-17 09:42:33.059276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-17 09:42:34.187480: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-17 09:42:34.187654: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-17 09:42:34.187658: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv



This error most likely means that this notebook is not configured to use a GPU.  Change this in Notebook Settings via the command palette (cmd/ctrl-shift-P) or the Edit menu.




SystemError: GPU device not found

In [2]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

# Load the CIFAR-10 dataset
data_path = 'cifar-10-batches-py/'
train_data = []
train_labels = []
for i in range(1, 6):
    batch = unpickle(data_path + 'data_batch_' + str(i))
    train_data.append(batch[b'data'])
    train_labels += batch[b'labels']
    
    
train_data = np.concatenate(train_data)
train_data = train_data.reshape((50000, 3, 32, 32)).transpose(0, 2, 3, 1)
test_data = unpickle(data_path + 'test_batch')[b'data']
test_data = test_data.reshape((10000, 3, 32, 32)).transpose(0, 2, 3, 1)
train_labels = np.array(train_labels)
test_labels = np.array(unpickle(data_path + 'test_batch')[b'labels'])

# Print the number of features, classes, and training samples for CIFAR-10
print("CIFAR-10")
print("Number of features:", train_data.shape[1:])
print("Number of classes:", len(np.unique(train_labels)))
print("Number of training samples:", train_data.shape[0])

# Shuffle your dataset
train_data, train_labels = shuffle(train_data, train_labels, random_state=42)
test_data, test_labels = shuffle(test_data, test_labels, random_state=42)

# Normalize pixel values
x_train = train_data.astype('float32') / 255.0
x_test = test_data.astype('float32') / 255.0

# Convert labels to categorical format
y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

# create data generator
datagen = ImageDataGenerator(zoom_range=[0.9,1.1],
                             rotation_range=30,
                             brightness_range=[0.9,1.1],
                             width_shift_range=0.3, 
                             height_shift_range=0.3,
                             vertical_flip=True,
                             horizontal_flip=True)
# prepare iterator
it_train = datagen.flow(x_train, y_train, batch_size=64)
# fit model
steps = int(x_train.shape[0] / 64)

CIFAR-10
Number of features: (32, 32, 3)
Number of classes: 10
Number of training samples: 50000


In [5]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

model_10.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI.h5', save_freq='epoch')

# compile model
opt = SGD(lr=0.001, momentum=0.9)
model_10.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 16, 16, 64)        36928     
                                                      

In [6]:
# Model Fit
model_10.fit(x_train,y_train,batch_size=64, epochs=400, validation_data=(x_test, y_test),callbacks=[checkpoint_callback])

Epoch 1/400
782/782 [==============================] - 5s 6ms/step - loss: 2.2784 - accuracy: 0.1420 - val_loss: 2.1875 - val_accuracy: 0.2019
Epoch 2/400
782/782 [==============================] - 4s 5ms/step - loss: 2.0951 - accuracy: 0.2091 - val_loss: 1.9812 - val_accuracy: 0.2808
Epoch 3/400
782/782 [==============================] - 4s 5ms/step - loss: 1.9535 - accuracy: 0.2550 - val_loss: 1.8115 - val_accuracy: 0.3392
Epoch 4/400
782/782 [==============================] - 4s 5ms/step - loss: 1.8145 - accuracy: 0.3055 - val_loss: 1.6783 - val_accuracy: 0.3879
Epoch 5/400
782/782 [==============================] - 4s 5ms/step - loss: 1.7319 - accuracy: 0.3442 - val_loss: 1.5773 - val_accuracy: 0.4230
Epoch 6/400
782/782 [==============================] - 4s 5ms/step - loss: 1.6721 - accuracy: 0.3681 - val_loss: 1.5248 - val_accuracy: 0.4430
Epoch 7/400
782/782 [==============================] - 4s 5ms/step - loss: 1.6134 - accuracy: 0.3972 - val_loss: 1.4887 - val_accuracy: 0.4596

Epoch 58/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7508 - accuracy: 0.7382 - val_loss: 0.6906 - val_accuracy: 0.7582
Epoch 59/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7419 - accuracy: 0.7410 - val_loss: 0.6445 - val_accuracy: 0.7711
Epoch 60/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7348 - accuracy: 0.7451 - val_loss: 0.6373 - val_accuracy: 0.7789
Epoch 61/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7308 - accuracy: 0.7473 - val_loss: 0.6453 - val_accuracy: 0.7728
Epoch 62/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7272 - accuracy: 0.7478 - val_loss: 0.6464 - val_accuracy: 0.7751
Epoch 63/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7170 - accuracy: 0.7521 - val_loss: 0.6387 - val_accuracy: 0.7792
Epoch 64/400
782/782 [==============================] - 4s 5ms/step - loss: 0.7115 - accuracy: 0.7529 - val_loss: 0.6316 - val_accuracy:

Epoch 115/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5228 - accuracy: 0.8218 - val_loss: 0.5215 - val_accuracy: 0.8253
Epoch 116/400
782/782 [==============================] - 4s 6ms/step - loss: 0.5191 - accuracy: 0.8200 - val_loss: 0.5311 - val_accuracy: 0.8223
Epoch 117/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5203 - accuracy: 0.8200 - val_loss: 0.5388 - val_accuracy: 0.8182
Epoch 118/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5113 - accuracy: 0.8212 - val_loss: 0.5261 - val_accuracy: 0.8232
Epoch 119/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5132 - accuracy: 0.8227 - val_loss: 0.5072 - val_accuracy: 0.8262
Epoch 120/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5067 - accuracy: 0.8257 - val_loss: 0.5087 - val_accuracy: 0.8299
Epoch 121/400
782/782 [==============================] - 4s 5ms/step - loss: 0.5108 - accuracy: 0.8251 - val_loss: 0.5161 - val_ac

782/782 [==============================] - 4s 5ms/step - loss: 0.3341 - accuracy: 0.8830 - val_loss: 0.4721 - val_accuracy: 0.8526
Epoch 228/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3381 - accuracy: 0.8831 - val_loss: 0.4710 - val_accuracy: 0.8512
Epoch 229/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3332 - accuracy: 0.8827 - val_loss: 0.4942 - val_accuracy: 0.8464
Epoch 230/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3329 - accuracy: 0.8854 - val_loss: 0.4672 - val_accuracy: 0.8542
Epoch 231/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3328 - accuracy: 0.8846 - val_loss: 0.4885 - val_accuracy: 0.8475
Epoch 232/400
782/782 [==============================] - 4s 6ms/step - loss: 0.3282 - accuracy: 0.8871 - val_loss: 0.4669 - val_accuracy: 0.8523
Epoch 233/400
782/782 [==============================] - 4s 5ms/step - loss: 0.3280 - accuracy: 0.8868 - val_loss: 0.4967 - val_accuracy: 0.8462

782/782 [==============================] - 4s 5ms/step - loss: 0.2493 - accuracy: 0.9133 - val_loss: 0.4782 - val_accuracy: 0.8619
Epoch 340/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2502 - accuracy: 0.9136 - val_loss: 0.5124 - val_accuracy: 0.8550
Epoch 341/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2506 - accuracy: 0.9130 - val_loss: 0.5148 - val_accuracy: 0.8540
Epoch 342/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2473 - accuracy: 0.9137 - val_loss: 0.5029 - val_accuracy: 0.8578
Epoch 343/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2470 - accuracy: 0.9133 - val_loss: 0.4905 - val_accuracy: 0.8583
Epoch 344/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2458 - accuracy: 0.9142 - val_loss: 0.4893 - val_accuracy: 0.8613
Epoch 345/400
782/782 [==============================] - 4s 5ms/step - loss: 0.2442 - accuracy: 0.9140 - val_loss: 0.4781 - val_accuracy: 0.8618

In [7]:

# load the pre-saved model
model_10 = load_model('model_10_gpu_Base-CubeAI.h5')

# test the accuracy of the model on the test set
loss, accuracy = model_10.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('Test accuracy:', accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 0.4928 - accuracy: 0.8610
Test accuracy: 0.8610000014305115


In [12]:
# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model_10)

# Convert the model to the TensorFlow Lite format with quantization
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

tflite_model_name = 'model_10_gpu_Base-CubeAI'
        
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open(tflite_model_name + '.tflite', 'wb').write(tflite_model)

tflite_interpreter = tf.lite.Interpreter(model_path=tflite_model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmp0abixuph/assets


INFO:tensorflow:Assets written to: /tmp/tmp0abixuph/assets
/home/themandalorian/anaconda3/envs/ml-on-mc/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-16 02:50:40.054263: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-16 02:50:40.054290: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-16 02:50:40.054965: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp0abixuph
2023-05-16 02:50:40.058443: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-16 02:50:40.058466: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp0abixuph
2023-05-16 02:50:40.066049: I tensorflow/

== Input details ==
name: serving_default_conv2d_6_input:0
shape: [ 1 32 32  3]
type: <class 'numpy.int8'>

== Output details ==
name: StatefulPartitionedCall:0
shape: [ 1 10]
type: <class 'numpy.int8'>
Accuracy of quantized to int8 model is 86.17%


In [20]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Convert the model to a quantization aware model
quant_aware_model = tfmot.quantization.keras.quantize_model(model_10)

quant_aware_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

quant_aware_model.summary()

# Train and evaluate the quantization aware model
quant_aware_model.fit(x_train,y_train, batch_size=64,epochs=400,validation_data=(x_test, y_test))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_6 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_36 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_conv2d_37 (QuantizeWr  (None, 32, 32, 32)       9315      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_18 (Qua  (None, 16, 16, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_24 (QuantizeW  (None, 16, 16, 32)      

782/782 [==============================] - 12s 15ms/step - loss: 0.6168 - accuracy: 0.7884 - val_loss: 0.6121 - val_accuracy: 0.7936
Epoch 88/400
782/782 [==============================] - 12s 16ms/step - loss: 0.6060 - accuracy: 0.7902 - val_loss: 0.5933 - val_accuracy: 0.7999
Epoch 89/400
782/782 [==============================] - 12s 15ms/step - loss: 0.6012 - accuracy: 0.7918 - val_loss: 0.6033 - val_accuracy: 0.7965
Epoch 90/400
782/782 [==============================] - 12s 15ms/step - loss: 0.6051 - accuracy: 0.7930 - val_loss: 0.5679 - val_accuracy: 0.8097
Epoch 91/400
782/782 [==============================] - 12s 15ms/step - loss: 0.6014 - accuracy: 0.7936 - val_loss: 0.5844 - val_accuracy: 0.8078
Epoch 92/400
782/782 [==============================] - 12s 16ms/step - loss: 0.5969 - accuracy: 0.7953 - val_loss: 0.5596 - val_accuracy: 0.8132
Epoch 93/400
782/782 [==============================] - 12s 16ms/step - loss: 0.5932 - accuracy: 0.7979 - val_loss: 0.5793 - val_accuracy

Epoch 143/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4577 - accuracy: 0.8439 - val_loss: 0.5149 - val_accuracy: 0.8295
Epoch 144/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4603 - accuracy: 0.8406 - val_loss: 0.4978 - val_accuracy: 0.8376
Epoch 145/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4614 - accuracy: 0.8421 - val_loss: 0.4880 - val_accuracy: 0.8423
Epoch 146/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4508 - accuracy: 0.8462 - val_loss: 0.4892 - val_accuracy: 0.8375
Epoch 147/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4538 - accuracy: 0.8428 - val_loss: 0.5082 - val_accuracy: 0.8376
Epoch 148/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4539 - accuracy: 0.8432 - val_loss: 0.5314 - val_accuracy: 0.8309
Epoch 149/400
782/782 [==============================] - 12s 16ms/step - loss: 0.4489 - accuracy: 0.8449 - val_loss: 0

Epoch 199/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3723 - accuracy: 0.8717 - val_loss: 0.4962 - val_accuracy: 0.8453
Epoch 200/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3736 - accuracy: 0.8710 - val_loss: 0.4805 - val_accuracy: 0.8450
Epoch 201/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3692 - accuracy: 0.8731 - val_loss: 0.4871 - val_accuracy: 0.8486
Epoch 202/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3671 - accuracy: 0.8724 - val_loss: 0.5146 - val_accuracy: 0.8423
Epoch 203/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3685 - accuracy: 0.8716 - val_loss: 0.4819 - val_accuracy: 0.8498
Epoch 204/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3628 - accuracy: 0.8726 - val_loss: 0.5165 - val_accuracy: 0.8438
Epoch 205/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3600 - accuracy: 0.8772 - val_loss: 0

Epoch 255/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3065 - accuracy: 0.8932 - val_loss: 0.4966 - val_accuracy: 0.8482
Epoch 256/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3065 - accuracy: 0.8946 - val_loss: 0.4974 - val_accuracy: 0.8486
Epoch 257/400
782/782 [==============================] - 12s 15ms/step - loss: 0.3070 - accuracy: 0.8916 - val_loss: 0.4828 - val_accuracy: 0.8529
Epoch 258/400
782/782 [==============================] - 12s 16ms/step - loss: 0.3054 - accuracy: 0.8945 - val_loss: 0.5114 - val_accuracy: 0.8464
Epoch 259/400
782/782 [==============================] - 12s 16ms/step - loss: 0.3045 - accuracy: 0.8943 - val_loss: 0.4934 - val_accuracy: 0.8543
Epoch 260/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2994 - accuracy: 0.8944 - val_loss: 0.4833 - val_accuracy: 0.8549
Epoch 261/400
782/782 [==============================] - 12s 16ms/step - loss: 0.3002 - accuracy: 0.8964 - val_loss: 0

Epoch 311/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2624 - accuracy: 0.9087 - val_loss: 0.4923 - val_accuracy: 0.8565
Epoch 312/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2633 - accuracy: 0.9085 - val_loss: 0.4909 - val_accuracy: 0.8561
Epoch 313/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2643 - accuracy: 0.9073 - val_loss: 0.4938 - val_accuracy: 0.8564
Epoch 314/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2624 - accuracy: 0.9079 - val_loss: 0.4882 - val_accuracy: 0.8592
Epoch 315/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2664 - accuracy: 0.9064 - val_loss: 0.5133 - val_accuracy: 0.8551
Epoch 316/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2657 - accuracy: 0.9088 - val_loss: 0.4892 - val_accuracy: 0.8579
Epoch 317/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2606 - accuracy: 0.9082 - val_loss: 0

Epoch 367/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2373 - accuracy: 0.9183 - val_loss: 0.5072 - val_accuracy: 0.8600
Epoch 368/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2343 - accuracy: 0.9183 - val_loss: 0.5145 - val_accuracy: 0.8561
Epoch 369/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2381 - accuracy: 0.9183 - val_loss: 0.4895 - val_accuracy: 0.8607
Epoch 370/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2275 - accuracy: 0.9204 - val_loss: 0.5184 - val_accuracy: 0.8553
Epoch 371/400
782/782 [==============================] - 12s 16ms/step - loss: 0.2304 - accuracy: 0.9194 - val_loss: 0.5163 - val_accuracy: 0.8593
Epoch 372/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2300 - accuracy: 0.9205 - val_loss: 0.4915 - val_accuracy: 0.8605
Epoch 373/400
782/782 [==============================] - 12s 15ms/step - loss: 0.2288 - accuracy: 0.9206 - val_loss: 0

In [3]:
# evaluate the model on the test set
quant_loss, quant_acc = quant_aware_model.evaluate(x_test, y_test, verbose=0)
print('Quantization aware training loss: ', quant_loss)
print('Quantization aware training accuracy: ', quant_acc)

# convert the QAT model to a fully quantized model using TFLite
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(x_train).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant_int8_qat = converter.convert()

open("model_10_gpu_Base-CubeAI_qat_int8.tflite", "wb").write(tflite_model_quant_int8_qat)

tflite_interpreter = tf.lite.Interpreter("model_10_gpu_Base-CubeAI_qat_int8.tflite")
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

predictions = np.zeros((len(x_test),), dtype=int)
input_scale, input_zero_point = input_details[0]["quantization"]
for i in range(len(x_test)):
    val_batch = x_test[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis=0).astype(input_details[0]["dtype"])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()

    tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
    #print("Prediction results shape:", tflite_model_predictions.shape)
    output = tflite_interpreter.get_tensor(output_details[0]['index'])
    predictions[i] = output.argmax()
    
sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

NameError: name 'quant_aware_model' is not defined

In [16]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

opt = SGD(lr=0.001, momentum=0.9)

# Strucutred pruning with constant sparsity
pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=2000, frequency=100),
        'block_size': (1, 1),
}

# Create a pruning model
pruned_model = tfmot.sparsity.keras.prune_low_magnitude(model_10, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

pruned_model.summary()

pruned_model_callback = tfmot.sparsity.keras.UpdatePruningStep()

# define the ModelCheckpoint callback to save the model after each epoch
pruned_checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_Structured_Pruning.h5', save_freq='epoch')

# Train and evaluate the pruned model
# if you get an error like train_function -> train_function
# just run the cell again
pruned_model.fit(x_train,y_train, batch_size=64,epochs=100,validation_data=(x_test, y_test),
                                      callbacks=[pruned_model_callback,pruned_checkpoint_callback])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       1762      
 18 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       18466     
 19 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_poo  (None, 16, 16, 32)       1         
 ling2d_9 (PruneLowMagnitude                                     
 )                                                               
                                                                 
 prune_low_magnitude_dropout  (None, 16, 16, 32)       1         
 _8 (PruneLowMagnitude)                                          
                                                      

782/782 [==============================] - 55s 70ms/step - loss: 0.4417 - accuracy: 0.8483 - val_loss: 0.5092 - val_accuracy: 0.8365
Epoch 88/100
782/782 [==============================] - 54s 68ms/step - loss: 0.4318 - accuracy: 0.8498 - val_loss: 0.5554 - val_accuracy: 0.8220
Epoch 89/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4401 - accuracy: 0.8510 - val_loss: 0.5415 - val_accuracy: 0.8272
Epoch 90/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4299 - accuracy: 0.8523 - val_loss: 0.5756 - val_accuracy: 0.8185
Epoch 91/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4286 - accuracy: 0.8533 - val_loss: 0.5390 - val_accuracy: 0.8288
Epoch 92/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4262 - accuracy: 0.8537 - val_loss: 0.5305 - val_accuracy: 0.8298
Epoch 93/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4262 - accuracy: 0.8530 - val_loss: 0.5253 - val_accuracy

In [19]:
# test the accuracy of the model on the test set
loss, accuracy = pruned_model.evaluate(x_test, y_test)

# print the accuracy of the model on the test set
print('pruned_model Test accuracy:', accuracy)

313/313 [==============================] - 2s 8ms/step - loss: 0.5212 - accuracy: 0.8349
pruned_model Test accuracy: 0.8349000215530396


In [25]:
pruned_model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)

pruned_keras_file = 'keras_model_10_gpu_Base-CubeAI_Structured_Pruning.h5'
tf.keras.models.save_model(pruned_model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', os.path.abspath(pruned_keras_file))

converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model_for_export)
pruned_tflite_model = converter.convert()

pruned_tflite_file = 'model_10_gpu_Base-CubeAI_Structured_Pruning_NoQ.tflite'

with open(pruned_tflite_file, 'wb') as f:
    f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', os.path.abspath(pruned_tflite_file))

test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_tflite_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_gpu_Base-CubeAI_Structured_Pruning_NoQ.tflite is {}%".format(accuracy_score*100))


Saved pruned Keras model to: /home/themandalorian/Git/ML-on-Microcontrollers/keras_model_10_gpu_Base-CubeAI_Structured_Pruning.h5


INFO:tensorflow:Assets written to: /tmp/tmp44k2fbv7/assets


INFO:tensorflow:Assets written to: /tmp/tmp44k2fbv7/assets
2023-05-16 15:15:36.888402: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-16 15:15:36.888427: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-16 15:15:36.888569: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp44k2fbv7
2023-05-16 15:15:36.890638: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-16 15:15:36.890651: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp44k2fbv7
2023-05-16 15:15:36.895897: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-05-16 15:15:36.919967: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp44k2fbv7
2023-05-16 15:15:36.928850: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_gpu_Base-CubeAI_Structured_Pruning_NoQ.tflite
Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of model_10_gpu_Base-CubeAI_Structured_Pruning_NoQ.tflite is 83.49%


In [27]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Unstrucutred pruning with constant sparsity
pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=2000, frequency=100),
}

# Create a pruning model
pruned_model_unstructured = tfmot.sparsity.keras.prune_low_magnitude(model_10, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model_unstructured.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_model_unstructured.summary()

pruned_model_callback = tfmot.sparsity.keras.UpdatePruningStep()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_Unstructured_Pruning.h5', save_freq='epoch')

# Train and evaluate the pruned model
pruned_model_unstructured.fit(x_train,y_train, batch_size=64,epochs=100,validation_data=(x_test, y_test),
                                      callbacks=[pruned_model_callback,checkpoint_callback])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       1762      
 30 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       18466     
 31 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_poo  (None, 16, 16, 32)       1         
 ling2d_15 (PruneLowMagnitud                                     
 e)                                                              
                                                                 
 prune_low_magnitude_dropout  (None, 16, 16, 32)       1         
 _16 (PruneLowMagnitude)                                         
                                                      

782/782 [==============================] - 54s 69ms/step - loss: 0.4343 - accuracy: 0.8493 - val_loss: 0.5672 - val_accuracy: 0.8249
Epoch 88/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4344 - accuracy: 0.8489 - val_loss: 0.5135 - val_accuracy: 0.8355
Epoch 89/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4381 - accuracy: 0.8499 - val_loss: 0.5358 - val_accuracy: 0.8310
Epoch 90/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4329 - accuracy: 0.8499 - val_loss: 0.5131 - val_accuracy: 0.8383
Epoch 91/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4335 - accuracy: 0.8501 - val_loss: 0.5643 - val_accuracy: 0.8288
Epoch 92/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4279 - accuracy: 0.8522 - val_loss: 0.5399 - val_accuracy: 0.8298
Epoch 93/100
782/782 [==============================] - 54s 69ms/step - loss: 0.4258 - accuracy: 0.8518 - val_loss: 0.5207 - val_accuracy

In [28]:
# Conversion to TF Lite
pruned_model_unstructured_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model_unstructured)

pruned_keras_file_unstructured = 'keras_model_10_gpu_Base-CubeAI_Unstructured_Pruning.h5'
tf.keras.models.save_model(pruned_model_unstructured_for_export, pruned_keras_file_unstructured, include_optimizer=False)
print('Saved pruned Keras model to:', os.path.abspath(pruned_keras_file_unstructured))

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model_unstructured_for_export)
pruned_tflite_model_unstructured = converter.convert()

# Save the model
pruned_tflite_file_unstructured = 'model_10_gpu_Base-CubeAI_Unstructured_Pruning.tflite'

with open(pruned_tflite_file_unstructured, 'wb') as f:
    f.write(pruned_tflite_model_unstructured)

print('Saved pruned TFLite model to:', os.path.abspath(pruned_tflite_file_unstructured))

test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_tflite_file_unstructured))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of pruned tflite unstructured model is {}%".format(accuracy_score*100))

Saved pruned Keras model to: /home/themandalorian/Git/ML-on-Microcontrollers/keras_model_10_gpu_Base-CubeAI_Unstructured_Pruning.h5


INFO:tensorflow:Assets written to: /tmp/tmpob7wxnar/assets


INFO:tensorflow:Assets written to: /tmp/tmpob7wxnar/assets
2023-05-16 23:31:30.408076: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-16 23:31:30.408110: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-16 23:31:30.408265: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpob7wxnar
2023-05-16 23:31:30.413161: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-16 23:31:30.413177: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpob7wxnar
2023-05-16 23:31:30.420324: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-05-16 23:31:30.446493: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpob7wxnar
2023-05-16 23:31:30.456173: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_gpu_Base-CubeAI_Unstructured_Pruning.tflite
Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of pruned tflite unstructured model is 83.36%


In [29]:
model_10 = Sequential()
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', 
                    input_shape=(32, 32, 3)))
model_10.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.2))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((2, 2)))
model_10.add(Dropout(0.3))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model_10.add(MaxPooling2D((4, 4)))
model_10.add(Dropout(0.4))
model_10.add(Flatten())
model_10.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model_10.add(Dropout(0.5))
model_10.add(Dense(10, activation='softmax'))

# Unstructured pruning with dynamic sparsity
pruning_params = {
        'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                                    final_sparsity=0.80,
                                                                    begin_step=2000,
                                                                    end_step=4000,
                                                                    frequency=100)

}

# Create a pruning model
pruned_model_unstructured_dynamic = tfmot.sparsity.keras.prune_low_magnitude(model_10, **pruning_params)

# `prune_low_magnitude` requires a recompile.
pruned_model_unstructured_dynamic.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_model_unstructured_dynamic.summary()

pruned_model_callback = tfmot.sparsity.keras.UpdatePruningStep()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_Dynamic_Unstructured_Pruning.h5', 
                                      save_freq='epoch')


pruned_model_unstructured_dynamic.fit(x_train,y_train, batch_size=64,epochs=100,validation_data=(x_test, y_test),
                                      callbacks=[pruned_model_callback,checkpoint_callback])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       1762      
 36 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_conv2d_  (None, 32, 32, 32)       18466     
 37 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_poo  (None, 16, 16, 32)       1         
 ling2d_18 (PruneLowMagnitud                                     
 e)                                                              
                                                                 
 prune_low_magnitude_dropout  (None, 16, 16, 32)       1         
 _20 (PruneLowMagnitude)                                         
                                                      

782/782 [==============================] - 54s 70ms/step - loss: 0.6584 - accuracy: 0.7744 - val_loss: 0.6165 - val_accuracy: 0.7960
Epoch 88/100
782/782 [==============================] - 54s 70ms/step - loss: 0.6612 - accuracy: 0.7759 - val_loss: 0.5944 - val_accuracy: 0.7966
Epoch 89/100
782/782 [==============================] - 54s 70ms/step - loss: 0.6582 - accuracy: 0.7749 - val_loss: 0.5749 - val_accuracy: 0.8054
Epoch 90/100
782/782 [==============================] - 54s 70ms/step - loss: 0.6483 - accuracy: 0.7775 - val_loss: 0.5889 - val_accuracy: 0.7980
Epoch 91/100
782/782 [==============================] - 54s 70ms/step - loss: 0.6549 - accuracy: 0.7757 - val_loss: 0.5928 - val_accuracy: 0.7974
Epoch 92/100
782/782 [==============================] - 54s 70ms/step - loss: 0.6535 - accuracy: 0.7732 - val_loss: 0.6130 - val_accuracy: 0.7899
Epoch 93/100
782/782 [==============================] - 54s 70ms/step - loss: 0.6546 - accuracy: 0.7762 - val_loss: 0.5743 - val_accuracy

In [30]:
# Conversion to Keras Lite
pruned_model_unstructured_dynamic_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model_unstructured_dynamic)

pruned_keras_file_unstructured_dynamic = 'keras_model_10_gpu_Base-CubeAI_Dynamic_Unstructured_Pruning.h5'
tf.keras.models.save_model(pruned_model_unstructured_dynamic_for_export, pruned_keras_file_unstructured_dynamic, include_optimizer=False)
print('Saved pruned Keras model to:', os.path.abspath(pruned_keras_file_unstructured_dynamic))

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model_unstructured_dynamic_for_export)
pruned_tflite_model_unstructured_dynamic = converter.convert()

# Save the model
pruned_tflite_file_unstructured_dynamic = 'model_10_gpu_Base-CubeAI_Dynamic_Unstructured_Pruning.tflite'

with open(pruned_tflite_file_unstructured_dynamic, 'wb') as f:
    f.write(pruned_tflite_model_unstructured_dynamic)

print('Saved pruned TFLite model to:', os.path.abspath(pruned_tflite_file_unstructured_dynamic))

test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_tflite_file_unstructured_dynamic))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of pruned tflite unstructured dynamic is {}%".format(accuracy_score*100))

Saved pruned Keras model to: /home/themandalorian/Git/ML-on-Microcontrollers/keras_model_10_gpu_Base-CubeAI_Dynamic_Unstructured_Pruning.h5


INFO:tensorflow:Assets written to: /tmp/tmpi3efuhzq/assets


INFO:tensorflow:Assets written to: /tmp/tmpi3efuhzq/assets
2023-05-17 09:04:50.424549: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 09:04:50.424872: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 09:04:50.426942: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpi3efuhzq
2023-05-17 09:04:50.429126: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 09:04:50.429140: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmpi3efuhzq
2023-05-17 09:04:50.441257: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-05-17 09:04:50.473017: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmpi3efuhzq
2023-05-17 09:04:50.481831: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

Saved pruned TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_gpu_Base-CubeAI_Dynamic_Unstructured_Pruning.tflite
Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of pruned tflite unstructured dynamic is 79.47%


In [6]:
pruned_model_for_export = load_model('keras_model_10_gpu_Base-CubeAI_Structured_Pruning.h5')

quant_aware_annotate_model = tfmot.quantization.keras.quantize_annotate_model(
              pruned_model_for_export)

pruned_qat_model = tfmot.quantization.keras.quantize_apply(quant_aware_annotate_model,
                   tfmot.experimental.combine.Default8BitPrunePreserveQuantizeScheme())

pruned_qat_model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_qat_model.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_Structured_QAT_Pruning.h5', save_freq='epoch')

# train the model
pruned_qat_model.fit(x_train,y_train, batch_size=64,epochs=10,validation_data=(x_test, y_test),
                                      callbacks=[checkpoint_callback])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_3 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_18 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_conv2d_19 (QuantizeWr  (None, 32, 32, 32)       9315      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_9 (Quan  (None, 16, 16, 32)       1         
 tizeWrapperV2)                                                  
                                                                 
 quant_dropout_8 (QuantizeWr  (None, 16, 16, 32)      

In [8]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_qat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
pruned_qat_tflite_model = converter.convert()

# Save the model
pruned_qat_tflite_file = 'model_10_gpu_Base-CubeAI_Structured_QAT_Pruning.tflite'

with open(pruned_qat_tflite_file, 'wb') as f:
    f.write(pruned_qat_tflite_model)

print('Saved pruned QAT TFLite model to:', os.path.abspath(pruned_qat_tflite_file))

test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(pruned_qat_tflite_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmp2hcafwg_/assets


INFO:tensorflow:Assets written to: /tmp/tmp2hcafwg_/assets
/home/themandalorian/anaconda3/envs/mlonmcu_lab2/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 10:18:07.413967: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 10:18:07.413997: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 10:18:07.414549: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp2hcafwg_
2023-05-17 10:18:07.420880: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 10:18:07.420909: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp2hcafwg_
2023-05-17 10:18:07.437040: I tensorf

Saved pruned QAT TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_gpu_Base-CubeAI_Structured_QAT_Pruning.tflite
Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of quantized to int8 model is 83.45%


In [9]:
pruned_model_unstructured_for_export = load_model('keras_model_10_gpu_Base-CubeAI_Unstructured_Pruning.h5')

quant_aware_annotate_model = tfmot.quantization.keras.quantize_annotate_model(pruned_model_unstructured_for_export)

pruned_qat_model_unstructured = tfmot.quantization.keras.quantize_apply(quant_aware_annotate_model,
                   tfmot.experimental.combine.Default8BitPrunePreserveQuantizeScheme())

pruned_qat_model_unstructured.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_qat_model_unstructured.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_Unstructured_QAT_Pruning.h5', save_freq='epoch')

# train the model
pruned_qat_model_unstructured.fit(x_train,y_train, batch_size=64,epochs=10,validation_data=(x_test, y_test),
                                      callbacks=[checkpoint_callback])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_5 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_30 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_conv2d_31 (QuantizeWr  (None, 32, 32, 32)       9315      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_15 (Qua  (None, 16, 16, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_16 (QuantizeW  (None, 16, 16, 32)      

In [11]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_qat_model_unstructured)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
unstructured_pruned_qat_tflite_model = converter.convert()

# Save the model
unstructured_pruned_qat_tflite_file = 'model_10_gpu_Base-CubeAI_Unstructured_QAT_Pruning.tflite'

with open(unstructured_pruned_qat_tflite_file, 'wb') as f:
    f.write(unstructured_pruned_qat_tflite_model)

print('Saved unstructured pruned QAT TFLite model to:', os.path.abspath(unstructured_pruned_qat_tflite_file))

test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(unstructured_pruned_qat_tflite_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of quantized to int8 model is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmplfxj5wmw/assets


INFO:tensorflow:Assets written to: /tmp/tmplfxj5wmw/assets
/home/themandalorian/anaconda3/envs/mlonmcu_lab2/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 10:32:45.226290: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 10:32:45.226324: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 10:32:45.226470: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmplfxj5wmw
2023-05-17 10:32:45.232789: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 10:32:45.232808: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmplfxj5wmw
2023-05-17 10:32:45.251208: I tensorf

Saved unstructured pruned QAT TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_gpu_Base-CubeAI_Unstructured_QAT_Pruning.tflite
Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of quantized to int8 model is 82.64%


In [12]:
pruned_model_unstructured_dynamic_for_export = load_model(
    'keras_model_10_gpu_Base-CubeAI_Dynamic_Unstructured_Pruning.h5')

quant_aware_annotate_model = tfmot.quantization.keras.quantize_annotate_model(
              pruned_model_unstructured_dynamic_for_export)

pruned_qat_model_unstructured_dynamic = tfmot.quantization.keras.quantize_apply(quant_aware_annotate_model,
                   tfmot.experimental.combine.Default8BitPrunePreserveQuantizeScheme())

pruned_qat_model_unstructured_dynamic.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])

pruned_qat_model_unstructured_dynamic.summary()

# define the ModelCheckpoint callback to save the model after each epoch
checkpoint_callback = ModelCheckpoint(filepath='model_10_gpu_Base-CubeAI_Dynamic_Unstructured_QAT_Pruning.h5', save_freq='epoch')

# train the model
pruned_qat_model_unstructured_dynamic.fit(x_train,y_train, batch_size=64,epochs=10,validation_data=(x_test, y_test),
                                      callbacks=[checkpoint_callback])

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_6 (QuantizeL  (None, 32, 32, 3)        3         
 ayer)                                                           
                                                                 
 quant_conv2d_36 (QuantizeWr  (None, 32, 32, 32)       963       
 apperV2)                                                        
                                                                 
 quant_conv2d_37 (QuantizeWr  (None, 32, 32, 32)       9315      
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_18 (Qua  (None, 16, 16, 32)       1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_dropout_20 (QuantizeW  (None, 16, 16, 32)      

In [13]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_qat_model_unstructured_dynamic)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_unstructured_pruned_qat_tflite_model = converter.convert()

# Save the model
dynamic_unstructured_pruned_qat_tflite_file = 'model_10_gpu_Base-CubeAI_Dynamic_Unstructured_QAT_Pruning.tflite'

with open(dynamic_unstructured_pruned_qat_tflite_file, 'wb') as f:
    f.write(dynamic_unstructured_pruned_qat_tflite_model)

print('Saved unstructured pruned QAT TFLite model to:', os.path.abspath(dynamic_unstructured_pruned_qat_tflite_file))

test_image_indices = range(x_test.shape[0])

# Initialize the interpreter
interpreter = tf.lite.Interpreter(model_path=str(dynamic_unstructured_pruned_qat_tflite_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

predictions = np.zeros((len(test_image_indices),), dtype=int)
for i, test_image_index in enumerate(test_image_indices):
    test_image = x_test[test_image_index]
    test_label = y_test[test_image_index]

    if (test_image_index % 1000 == 0):
        print("Evaluated on %d images." % test_image_index)

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
        input_scale, input_zero_point = input_details["quantization"]
        test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

sum = 0
for i in range(len(predictions)):
    if (predictions[i] == test_labels[i]):
        sum = sum + 1
accuracy_score = sum / 10000
print("Accuracy of model_10_gpu_Base-CubeAI_Dynamic_Unstructured_QAT_Pruning.tflite is {}%".format(accuracy_score*100))

INFO:tensorflow:Assets written to: /tmp/tmp690etpvg/assets


INFO:tensorflow:Assets written to: /tmp/tmp690etpvg/assets
/home/themandalorian/anaconda3/envs/mlonmcu_lab2/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:765: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-05-17 10:45:30.394982: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-05-17 10:45:30.395017: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-05-17 10:45:30.395173: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp690etpvg
2023-05-17 10:45:30.407055: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-05-17 10:45:30.407087: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp690etpvg
2023-05-17 10:45:30.436444: I tensorf

Saved unstructured pruned QAT TFLite model to: /home/themandalorian/Git/ML-on-Microcontrollers/model_10_gpu_Base-CubeAI_Dynamic_Unstructured_QAT_Pruning.tflite
Evaluated on 0 images.
Evaluated on 1000 images.
Evaluated on 2000 images.
Evaluated on 3000 images.
Evaluated on 4000 images.
Evaluated on 5000 images.
Evaluated on 6000 images.
Evaluated on 7000 images.
Evaluated on 8000 images.
Evaluated on 9000 images.
Accuracy of model_10_gpu_Base-CubeAI_Dynamic_Unstructured_QAT_Pruning.tflite is 80.92%


In [14]:
# Function: Convert some hex value into an array for C programming
def hex_to_c_array(hex_data, var_name):

    c_str = ''

    # Create header guard
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += '#define ' + var_name.upper() + '_H\n\n'

    # Add array length at top of file
    c_str += '\nstatic const unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'

    # Declare C variable
    c_str += 'static const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data) :

        # Construct string from hex
        hex_str = format(val, '#04x')

        # Add formatting so each line stays within 80 characters
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n '
        hex_array.append(hex_str)

    # Add closing brace
    c_str += '\n ' + format(' '.join(hex_array)) + '\n};\n\n'

    # Close out header guard
    c_str += '#endif //' + var_name.upper() + '_H'

    return c_str

In [15]:
c_model_name = 'cifar10_pruned_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(pruned_qat_tflite_model, c_model_name))

In [16]:
c_model_name = 'cifar10_unstructured_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(unstructured_pruned_qat_tflite_model, c_model_name))

In [17]:
c_model_name = 'cifar10_dynamic_qat'
# check if dir 'cfiles' exists, if not create it
if not os.path.exists('cfiles'):
    os.makedirs('cfiles')
# Write TFLite model to a C source (or header) file
with open('cfiles/' + c_model_name + '.h', 'w') as file:
    file.write(hex_to_c_array(dynamic_unstructured_pruned_qat_tflite_model, c_model_name))